In [1]:
from google.adk.agents import Agent, SequentialAgent, ParallelAgent, LoopAgent
from google.adk.models.google_llm import Gemini
from google.adk.runners import InMemoryRunner
from google.adk.tools import AgentTool, FunctionTool, google_search
from google.genai import types

from google.adk.tools.mcp_tool.mcp_toolset import McpToolset
from google.adk.tools.tool_context import ToolContext
from google.adk.tools.mcp_tool.mcp_session_manager import StdioConnectionParams
from mcp import StdioServerParameters

from google.adk.apps.app import App, ResumabilityConfig
from google.adk.tools.function_tool import FunctionTool

from google.adk.sessions import DatabaseSessionService
from google.adk.memory import InMemoryMemoryService
from google.adk.tools import load_memory, preload_memory

from google.adk.runners import Runner
from google.adk.plugins.logging_plugin import (
    LoggingPlugin,
)

print("✅ ADK components imported successfully.")

✅ ADK components imported successfully.


In [2]:
import os
from kaggle_secrets import UserSecretsClient

try:
    GOOGLE_API_KEY = UserSecretsClient().get_secret("GOOGLE_API_KEY")
    os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY
    print("✅ Setup and authentication complete.")
except Exception as e:
    print(
        f"🔑 Authentication Error: Please make sure you have added 'GOOGLE_API_KEY' to your Kaggle secrets. Details: {e}"
    )

✅ Setup and authentication complete.


In [3]:
retry_config=types.HttpRetryOptions(
    attempts=5,  # Maximum retry attempts
    exp_base=7,  # Delay multiplier
    initial_delay=1,
    http_status_codes=[429, 500, 503, 504], # Retry on these HTTP errors
)

In [4]:
# function to read a file from a given location and return its content
def read_file(file_path: str) -> str:
    with open(file_path, 'r') as file:
        content = file.read()
    return content

In [5]:
# function to write content to a file at a given location
def write_file(file_path: str, content: str) -> None:
    """Writes the given content to a file at a given location"""
    with open(file_path, 'w') as file:
        file.write(content)

In [6]:
#function to fetch a URL and return the content
def fetch_url(url: str) -> str:
    import requests
    response = requests.get(url)
    response.raise_for_status()  # Raise an error for bad responses
    return response.text

In [7]:
async def auto_save_to_memory(callback_context):
    """Automatically save session to memory after each agent turn."""
    await callback_context._invocation_context.memory_service.add_session_to_memory(
        callback_context._invocation_context.session
    )

print("✅ Callback created.")

✅ Callback created.


In [8]:
APP_NAME = "default"  # Application
USER_ID = "default"  # User
SESSION = "default"  # Session

# Consolidated agent definitions with explicit input_key and output_key
# Root coordinator agent
root_agent = Agent(
    name="requirement_assistant",
    model="gemini-2.5-flash-lite",
    description="Coordinate tasks and collect requirements.",
    instruction=""""
        Collect the requirement and state the problem clearly.
        """,
    tools=[fetch_url, read_file],
    after_agent_callback=auto_save_to_memory,  # Saves after each turn!
    output_key="root_agent_requirements",
)
print("✅ root_agent defined.")

# Summarizer agent: summarizes research/findings into key points
summarizer_agent = Agent(
    name="SummarizerAgent",
    model=Gemini(model="gemini-2.5-flash-lite", retry_options=retry_config),
    instruction="Read the provided research findings and return 3-5 bulleted key points. (Requirements: {root_agent_requirements})",
    after_agent_callback=auto_save_to_memory,  # Saves after each turn!
    output_key="final_summary",
)
print("✅ summarizer_agent created.")

# Coding agent: writes candidate implementation based on findings and requirements
coding_agent = Agent(
    name="CodingAgent",
    model=Gemini(model="gemini-2.5-flash-lite", retry_options=retry_config),
    instruction="""
        Write concise, well-tested code and nothing else based on the provided requirements and research findings. 
        (Final Summary: {final_summary}, Root Agent Requirements: {root_agent_requirements})
        Write the code to a file called 'candidate_code.py'
        Do not attempt to run the file.
        """,
     tools=[write_file],
    output_key="candidate_code",
)
print("✅ coding_agent created.")

# Unit test agent: generates unit tests for the candidate code
test_agent = Agent(
    name="TestAgent",
    model=Gemini(model="gemini-2.5-flash-lite", retry_options=retry_config),
    instruction="""
    Generate unit tests that validate behaviors described by the requirements given here: {root_agent_requirements} 
    Use the read_file to read the candidate code named 'candidate_code.py'
    (Root Agent Requirements: {root_agent_requirements})
    Write the tests to a file called  'unit_tests.py'
    Do not attempt to run the file.
    """,
    tools=[preload_memory, write_file, read_file],
    output_key="unit_tests",
)
print("✅ test_agent created.")

# Functional test agent: produces high-level end-to-end tests
functional_test_agent = Agent(
    name="FunctionalTestAgent",
    model=Gemini(model="gemini-2.5-flash-lite", retry_options=retry_config),
    instruction="""
    Produce Playwright tests for the requirements given here: {root_agent_requirements} 
    Use the read_file to read the candidate code named 'candidate_code.py'
    user's request for Playwright tests
    Do not attempt to implement the requirement yourself.
    Write the Playwright tests  to a file called 'functional_tests.py
    Do not attempt to run the file.
    """,
    tools=[write_file, read_file],
    output_key="functional_tests",
)
print("✅ functional_test_agent created.")

# Mutation test agent: proposes mutation tests to evaluate test-suite strength
mutation_test_agent = Agent(
    name="MutationTestAgent",
    model=Gemini(model="gemini-2.5-flash-lite", retry_options=retry_config),
    instruction="""
    Write mutation tests using mutpy based on the candidate code. 
    Use the read_file tool to read the candidate code named 'candidate_code.py'
    Write the mutation tests to a file called 'mutation_tests.py'
    Do not attempt to run the file.
    """,
    tools=[preload_memory, write_file, read_file],
    output_key="mutation_test_findings",
)
print("✅ mutation_test_agent created.")

# Property-based test agent: generates QuickCheck-style properties
property_based_test_agent = Agent(
    name="PropertyBasedTestAgent",
    model=Gemini(model="gemini-2.5-flash-lite", retry_options=retry_config),
    instruction="""
    Generate property-based tests  and nothing else using pytest from deterministic functions and functional tests. 
    Use the read_file tool to read the candidate code named 'candidate_code.py'
    (Functional Tests: {functional_tests})
    Write the property based tests to a file called 'property_based_tests.py'
    Do not attempt to run the file.
    """,
    tools=[preload_memory, write_file, read_file],
    output_key="property_based_test_findings",
)
print("✅ property_based_test_agent created.")

# Chaos engineering agent: designs resilience experiments
chaos_engineering_test_agent = Agent(
    name="ChaosEngineeringTestAgent",
    model=Gemini(model="gemini-2.5-flash-lite", retry_options=retry_config),
    instruction="""Write Tests for safe chaos experiments (network latency, resource pressure) and recovery checks  and nothing else. 
    Use the read_file tool to read the candidate code named 'candidate_code.py'
    Read the Unit Tests here and avoid witing those tests again: {unit_tests}, 
    Read the functional tests here: {functional_tests},
    Read the root agent requirements here: {root_agent_requirements}
    Write the chaos engineering tests to a file called 'chaos_engg_tests.py'
    Do not attempt to run the file.
    """,
    tools=[preload_memory, write_file, read_file],
    output_key="chaos_engineering_tests",
)
print("✅ chaos_engineering_test_agent created.")

# Adversarial/robustness agent: creates defensive adversarial tests and mitigations
adversarial_test_agent = Agent(
    name="AdversarialTestAgent",
    model=Gemini(model="gemini-2.5-flash-lite", retry_options=retry_config),
    instruction="""
    Generate defensive adversarial test cases (fuzzing, malformed inputs, boundary checks) and suggest mitigations. 
    Do NOT generate exploit code. Also avoid repeating existing unit tests and functional tests. 
    Use the read_file tool to read the candidate code named 'candidate_code.py'
    Read the Unit Tests here and avoid witing those tests again: {unit_tests}, 
    Read the functional tests here: {functional_tests},
    Read the root agent requirements here: {root_agent_requirements})
    Write the adversarial tests to a file called 'adversarial_tests.py'
    Do not attempt to run the file.
    """,
    tools=[preload_memory, write_file, read_file],
    output_key="adversarial_tests",
)
print("✅ adversarial_test_agent created.")

✅ root_agent defined.
✅ summarizer_agent created.
✅ coding_agent created.
✅ test_agent created.
✅ functional_test_agent created.
✅ mutation_test_agent created.
✅ property_based_test_agent created.
✅ chaos_engineering_test_agent created.
✅ adversarial_test_agent created.


In [9]:
# The ParallelAgent runs all its sub-agents simultaneously.
parallel_research_team = ParallelAgent(
    name="ParallelResearchTeam",
    sub_agents=[ chaos_engineering_test_agent, property_based_test_agent, mutation_test_agent ],
)

# This SequentialAgent defines the high-level workflow: run the parallel team first, then run the aggregator.
sequential_agent = SequentialAgent(
    name="ResearchSystem",
    sub_agents=[root_agent, summarizer_agent,  coding_agent, functional_test_agent, test_agent, parallel_research_team, adversarial_test_agent],
)

print("✅ Parallel and Sequential Agents created.")

✅ Parallel and Sequential Agents created.


In [10]:
# DatabaseSessionService
# SQLite database will be created automatically
db_url = "sqlite:///my_agent_data.db"  # Local SQLite file
session_service = DatabaseSessionService(db_url=db_url)

memory_service = (
    InMemoryMemoryService()
)  # ADK's built-in Memory Service for development and testing

In [11]:
# Step 3: Create a new runner with persistent storage
runner = Runner(
            agent=sequential_agent, 
            app_name=APP_NAME, 
            session_service=session_service,
            memory_service=memory_service, 
            plugins=[
                LoggingPlugin()
            ]
         )

print("✅ Upgraded to persistent sessions!")
print(f"   - Database: my_agent_data.db")
print(f"   - Sessions will survive restarts!")

✅ Upgraded to persistent sessions!
   - Database: my_agent_data.db
   - Sessions will survive restarts!


In [12]:
# Define run_session helper function that will be reused throughout the notebook
async def run_session(
    runner_instance: Runner,
    user_queries: list[str] | str = None,
    session_name: str = "default",
):
    print(f"\n ### Session: {session_name}")

    # Get app name from the Runner
    app_name = runner_instance.app_name

    # Attempt to create a new session or retrieve an existing one
    try:
        session = await session_service.create_session(
            app_name=app_name, user_id=USER_ID, session_id=session_name
        )
    except:
        session = await session_service.get_session(
            app_name=app_name, user_id=USER_ID, session_id=session_name
        )

    # Process queries if provided
    if user_queries:
        # Convert single query to list for uniform processing
        if type(user_queries) == str:
            user_queries = [user_queries]

        # Process each query in the list sequentially
        for query in user_queries:
            print(f"\nUser > {query}")

            # Convert the query string to the ADK Content format
            query = types.Content(role="user", parts=[types.Part(text=query)])

            # Stream the agent's response asynchronously
            async for event in runner_instance.run_async(
                user_id=USER_ID, session_id=session.id, new_message=query
            ):
                # Check if the event contains valid content
                if event.content and event.content.parts:
                    # Filter out empty or "None" responses before printing
                    if (
                        event.content.parts[0].text != "None"
                        and event.content.parts[0].text
                    ):
                        print(f"{MODEL_NAME} > ", event.content.parts[0].text)
    else:
        print("No queries!")


print("✅ Helper functions defined.")

✅ Helper functions defined.


In [13]:
MODEL_NAME = "gemini-2.5-flash-lite"
await run_session(
    runner,
    ["Write a simple calculator that accepts a text expression supporting basic arithmetic operations and prints back the result"],
    "stateful-agentic-session",
)
print("✅ Generation completed.")


 ### Session: stateful-agentic-session

User > Write a simple calculator that accepts a text expression supporting basic arithmetic operations and prints back the result
[logging_plugin] 🚀 USER MESSAGE RECEIVED
[logging_plugin]    Invocation ID: e-79e7a1dc-8fdf-40d0-84e4-3e9d4d944c5a
[logging_plugin]    Session ID: stateful-agentic-session
[logging_plugin]    User ID: default
[logging_plugin]    App Name: default
[logging_plugin]    Root Agent: ResearchSystem
[logging_plugin]    User Content: text: 'Write a simple calculator that accepts a text expression supporting basic arithmetic operations and prints back the result'
[logging_plugin] 🏃 INVOCATION STARTING
[logging_plugin]    Invocation ID: e-79e7a1dc-8fdf-40d0-84e4-3e9d4d944c5a
[logging_plugin]    Starting Agent: ResearchSystem
[logging_plugin] 🤖 AGENT STARTING
[logging_plugin]    Agent Name: ResearchSystem
[logging_plugin]    Invocation ID: e-79e7a1dc-8fdf-40d0-84e4-3e9d4d944c5a
[logging_plugin] 🤖 AGENT STARTING
[logging_plugin] 

[logging_plugin] 🧠 LLM RESPONSE
[logging_plugin]    Agent: CodingAgent
[logging_plugin]    Content: function_call: write_file
[logging_plugin]    Token Usage - Input: 512, Output: 299
[logging_plugin] 📢 EVENT YIELDED
[logging_plugin]    Event ID: e0d66d6b-5397-40d5-8a1c-cebbaffd0a5d
[logging_plugin]    Author: CodingAgent
[logging_plugin]    Content: function_call: write_file
[logging_plugin]    Final Response: False
[logging_plugin]    Function Calls: ['write_file']
[logging_plugin] 🔧 TOOL STARTING
[logging_plugin]    Tool Name: write_file
[logging_plugin]    Agent: CodingAgent
[logging_plugin]    Function Call ID: adk-b1f5498e-3596-4d0e-a028-16676f5a37e3
[logging_plugin]    Arguments: {'file_path': 'candidate_code.py', 'content': '\ndef calculate(expression):\n    """Parses and calculates the result of a simple arithmetic expression."""\n    try:\n        num1, operator, num2 = expression.split()\n        num1 = float(num1)\n        num2 = float(num2)\n\n        if operator == "+...}

[logging_plugin] 🧠 LLM RESPONSE
[logging_plugin]    Agent: FunctionalTestAgent
[logging_plugin]    Content: function_call: read_file
[logging_plugin]    Token Usage - Input: 835, Output: 21
[logging_plugin] 📢 EVENT YIELDED
[logging_plugin]    Event ID: ad614288-64e4-4c1e-849f-41c147525fb9
[logging_plugin]    Author: FunctionalTestAgent
[logging_plugin]    Content: function_call: read_file
[logging_plugin]    Final Response: False
[logging_plugin]    Function Calls: ['read_file']
[logging_plugin] 🔧 TOOL STARTING
[logging_plugin]    Tool Name: read_file
[logging_plugin]    Agent: FunctionalTestAgent
[logging_plugin]    Function Call ID: adk-463d16c8-a5ba-42d0-8fd9-f255ef44aa8a
[logging_plugin]    Arguments: {'file_path': 'candidate_code.py'}
[logging_plugin] 🔧 TOOL COMPLETED
[logging_plugin]    Tool Name: read_file
[logging_plugin]    Agent: FunctionalTestAgent
[logging_plugin]    Function Call ID: adk-463d16c8-a5ba-42d0-8fd9-f255ef44aa8a
[logging_plugin]    Result: 
def calculate(expres

[logging_plugin] 🧠 LLM RESPONSE
[logging_plugin]    Agent: FunctionalTestAgent
[logging_plugin]    Content: function_call: write_file
[logging_plugin]    Token Usage - Input: 1144, Output: 958
[logging_plugin] 📢 EVENT YIELDED
[logging_plugin]    Event ID: 2139a889-7df7-410d-aa4b-e7986117e21a
[logging_plugin]    Author: FunctionalTestAgent
[logging_plugin]    Content: function_call: write_file
[logging_plugin]    Final Response: False
[logging_plugin]    Function Calls: ['write_file']
[logging_plugin] 🔧 TOOL STARTING
[logging_plugin]    Tool Name: write_file
[logging_plugin]    Agent: FunctionalTestAgent
[logging_plugin]    Function Call ID: adk-933e55a8-2876-438d-a00d-cd0ed2e5d365
[logging_plugin]    Arguments: {'file_path': 'functional_tests.py', 'content': '\nfrom playwright.sync_api import Page, expect, sync_playwright\n\ndef test_calculator():\n    with sync_playwright() as p:\n        browser = p.chromium.launch()\n        page = browser.new_page()\n\n        # Test case 1: Additi

[logging_plugin] 🧠 LLM RESPONSE
[logging_plugin]    Agent: TestAgent
[logging_plugin]    Content: function_call: read_file | function_call: write_file
[logging_plugin]    Token Usage - Input: 2598, Output: 600
[logging_plugin] 📢 EVENT YIELDED
[logging_plugin]    Event ID: 7ada207e-7187-4702-946e-d89f2e198298
[logging_plugin]    Author: TestAgent
[logging_plugin]    Content: function_call: read_file | function_call: write_file
[logging_plugin]    Final Response: False
[logging_plugin]    Function Calls: ['read_file', 'write_file']
[logging_plugin] 🔧 TOOL STARTING
[logging_plugin]    Tool Name: read_file
[logging_plugin]    Agent: TestAgent
[logging_plugin]    Function Call ID: adk-310344bb-254f-4cd8-bec9-014e6bb27a05
[logging_plugin]    Arguments: {'file_path': 'candidate_code.py'}
[logging_plugin] 🔧 TOOL COMPLETED
[logging_plugin]    Tool Name: read_file
[logging_plugin]    Agent: TestAgent
[logging_plugin]    Function Call ID: adk-310344bb-254f-4cd8-bec9-014e6bb27a05
[logging_plugin] 

ExceptionGroup: unhandled errors in a TaskGroup (1 sub-exception)

In [ ]:
# Define ADK helper functions that will be reused throughout the notebook

from IPython.core.display import display, HTML
from jupyter_server.serverapp import list_running_servers

# Gets the proxied URL in the Kaggle Notebooks environment
def get_adk_proxy_url():
    PROXY_HOST = "https://kkb-production.jupyter-proxy.kaggle.net"
    ADK_PORT = "8000"

    servers = list(list_running_servers())
    if not servers:
        raise Exception("No running Jupyter servers found.")

    baseURL = servers[0]['base_url']

    try:
        path_parts = baseURL.split('/')
        kernel = path_parts[2]
        token = path_parts[3]
    except IndexError:
        raise Exception(f"Could not parse kernel/token from base URL: {baseURL}")

    url_prefix = f"/k/{kernel}/{token}/proxy/proxy/{ADK_PORT}"
    url = f"{PROXY_HOST}{url_prefix}"

    styled_html = f"""
    <div style="padding: 15px; border: 2px solid #f0ad4e; border-radius: 8px; background-color: #fef9f0; margin: 20px 0;">
        <div style="font-family: sans-serif; margin-bottom: 12px; color: #333; font-size: 1.1em;">
            <strong>⚠️ IMPORTANT: Action Required</strong>
        </div>
        <div style="font-family: sans-serif; margin-bottom: 15px; color: #333; line-height: 1.5;">
            The ADK web UI is <strong>not running yet</strong>. You must start it in the next cell.
            <ol style="margin-top: 10px; padding-left: 20px;">
                <li style="margin-bottom: 5px;"><strong>Run the next cell</strong> (the one with <code>!adk web ...</code>) to start the ADK web UI.</li>
                <li style="margin-bottom: 5px;">Wait for that cell to show it is "Running" (it will not "complete").</li>
                <li>Once it's running, <strong>return to this button</strong> and click it to open the UI.</li>
            </ol>
            <em style="font-size: 0.9em; color: #555;">(If you click the button before running the next cell, you will get a 500 error.)</em>
        </div>
        <a href='{url}' target='_blank' style="
            display: inline-block; background-color: #1a73e8; color: white; padding: 10px 20px;
            text-decoration: none; border-radius: 25px; font-family: sans-serif; font-weight: 500;
            box-shadow: 0 2px 5px rgba(0,0,0,0.2); transition: all 0.2s ease;">
            Open ADK Web UI (after running cell below) ↗
        </a>
    </div>
    """

    display(HTML(styled_html))

    return url_prefix

print("✅ Helper functions defined.")

In [ ]:
#!adk create sample-agent --model gemini-2.5-flash-lite --api_key $GOOGLE_API_KEY

In [ ]:
url_prefix = get_adk_proxy_url()

In [ ]:
#!adk web --url_prefix {url_prefix}

In [ ]:
session = await session_service.get_session(
    app_name=APP_NAME, user_id=USER_ID, session_id="conversation-01"
)

# Search for color preferences
search_response = await memory_service.search_memory(
    app_name=APP_NAME, user_id=USER_ID, query="What are the operations supported ?"
)

print("🔍 Search Results:")
print(f"  Found {len(search_response.memories)} relevant memories")
print(f"{search_response}")